In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [5]:
# 6.1 Identifying Missing Data
# We will use the orders table from Super Store for these tasks
orders = pd.read_csv('6_09_orders.csv')


# A. Start with some exploratory analysis methods to inspect the data
orders.head()
orders.info()
orders.columns
orders.shape
pd.DataFrame(orders.dtypes, columns=["DataTypes"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990677 entries, 0 to 990676
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   order_id     990677 non-null  object 
 1   order_date   990677 non-null  object 
 2   ship_date    990677 non-null  object 
 3   ship_mode    990677 non-null  object 
 4   customer_id  990677 non-null  object 
 5   product_id   990677 non-null  object 
 6   sales        990677 non-null  float64
 7   quantity     990677 non-null  int64  
 8   discount     990677 non-null  float64
 9   profit       990677 non-null  float64
 10  postal_code  191101 non-null  float64
 11  region_id    990484 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 90.7+ MB


,DataTypes
order_id,object
order_date,object
ship_date,object
ship_mode,object
customer_id,object
product_id,object
sales,float64
quantity,int64
discount,float64
profit,float64


In [7]:
# B. Which column has the most missing data? Sort the columns by sum of null values
orders.isnull().sum().sort_values(ascending=False)

postal_code    799576
region_id         193
order_id            0
order_date          0
ship_date           0
ship_mode           0
customer_id         0
product_id          0
sales               0
quantity            0
discount            0
profit              0
dtype: int64

In [9]:
# C. Looks like postal_code is our biggest problem, along with region_id
# Use a combination of filtering, isnull, and sum to count how many rows are missing both columns
orders[orders['postal_code'].isnull()]['region_id'].isnull().sum()

193

In [11]:
# D. Let's drop the region_id nulls from the dataframe before proceeding
orders.dropna(subset=['region_id'], inplace=True)

In [13]:
# E. It's the dream scenario! The IT team confirms all missing postal_code values should be 10001.0
orders['postal_code'].fillna(10001.0, inplace=True)
orders.isnull().sum()

/var/folders/1j/ld64vwsn607_t_vb5t8xfl_r0000gn/T/ipykernel_11052/2668897053.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  orders['postal_code'].fillna(10001.0, inplace=True)


order_id       0
order_date     0
ship_date      0
ship_mode      0
customer_id    0
product_id     0
sales          0
quantity       0
discount       0
profit         0
postal_code    0
region_id      0
dtype: int64

In [15]:
# 6.2 Cleaning Our Data
# A. Write a profit_margin function that accepts a row of data, which is a dictionary
#    It should return the result of dividing the profit column by the sales column (i.e. profit/sales)
def profit_margin(row):
    return row['profit'] / row['sales']

In [17]:
# B. Create a new column in the orders dataframe called 'profit_margin' by applying the profit margin function row-by-row
orders['profit_margin'] = orders.apply(profit_margin, axis=1)

# Note: an alternative way to do this without a function would be:
orders['profit_margin'] = orders['profit'] / orders['sales']

In [18]:
# C. Use the same process to create a new column called margin_category
# If the profit_margin is less than 0, the margin_category should be "unprofitable"
# If the profit_margin is 0, the margin_category should be "break even"
# If the profit_margin is above 0, the margin_category should "profitable"
def margin_categorization(row):
    if row['profit_margin'] > 0:
        return "profitable"
    elif row['profit_margin'] == 0:
        return "break even"
    else:
        return "unprofitable"
orders['margin_category'] = orders.apply(margin_categorization, axis=1)

In [19]:
# D. How many of our orders were unprofitable?
orders[orders['margin_category'] == "unprofitable"].shape[0]

11821

In [20]:
# 6.3 GroupBy Insights
# Segment the following data and explore aggregate values to answer the following questions

In [21]:
# A. Which discount results in the highest mean order quantity?
orders.groupby('discount')['quantity'].mean().sort_values(ascending=False).iloc[[0]].index[0]

0.63

In [27]:
# B. Which product has the highest mean price discount applied?
orders.groupby('product_id')['discount'].mean().sort_values(ascending=False).iloc[[0]].index[0]

'OFF-AP-10002899'

In [29]:
# 6.4 Joining DataFrames
# The below example joins our first two dataframes by their shared column, Symbol
openprice = pd.DataFrame({'Symbol': ['AAPL', 'DHR', 'DAL', 'AMZN'], 'OpenPrice': [217.51, 96.54, 51.45, 1703.34]})
wkhigh = pd.DataFrame({'Symbol': ['DAL', 'AMZN', 'AAPL', 'DHR'], '52wkHigh': [60.79, 2050.49, 233.47, 110.11]})
combined = pd.merge(openprice, wkhigh, how="left", left_on='Symbol', right_on='Symbol')
combined.head()

,Symbol,OpenPrice,52wkHigh
0,AAPL,217.51,233.47
1,DHR,96.54,110.11
2,DAL,51.45,60.79
3,AMZN,1703.34,2050.49


In [31]:
# A. Join the stockname dataframe to our combined result and print the result
stockname = pd.DataFrame({'Symbol': ['AMZN', 'DHR', 'DAL', 'AAPL'], 'Name': ['Amazon', 'Danaher', 'Delta Airlines', 'Apple']})
combined = pd.merge(combined, stockname, how="left", on="Symbol")
combined.head()

,Symbol,OpenPrice,52wkHigh,Name
0,AAPL,217.51,233.47,Apple
1,DHR,96.54,110.11,Danaher
2,DAL,51.45,60.79,Delta Airlines
3,AMZN,1703.34,2050.49,Amazon


In [33]:
# B. Use the following tables from Super Store
products = pd.read_csv('./datasets/products.csv')
orders = pd.read_csv('./datasets/orders.csv')
returns = pd.read_csv('./datasets/returns.csv')
regions = pd.read_csv('./datasets/regions.csv')

FileNotFoundError: [Errno 2] No such file or directory: './datasets/products.csv'

In [35]:
# i. We want to join the products and orders dataframes. 
#    Explore both dataframes to identify the common column between them
#    Use a left join to combine the tables in a dataframe named orders_with_products
orders_with_products = pd.merge(left=products, right=orders, how="left", on="product_id")
orders_with_products.columns

NameError: name 'products' is not defined

In [37]:
# ii. Left join the orders_with_products and returns dataframes
orders_with_products_and_returns = pd.merge(left=orders_with_products, right=returns, how="left", on="order_id")
orders_with_products_and_returns.columns

NameError: name 'orders_with_products' is not defined

In [39]:
# iii. Finally, add the region data to our combined dataframe
orders_with_products_and_returns_and_regions = pd.merge(left=orders_with_products_and_returns, right=regions, how="left", on="region_id")
orders_with_products_and_returns_and_regions.columns

NameError: name 'orders_with_products_and_returns' is not defined

In [41]:
# iv. Let's use this combined dataframe to determine the salesperson generating the most profit
orders_with_products_and_returns_and_regions.groupby('salesperson')['profit'].sum().sort_values(ascending=False).iloc[[0]].index[0]

NameError: name 'orders_with_products_and_returns_and_regions' is not defined